## Imports

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier, BayesianRidge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.neighbors import KNeighborsClassifier
from skmultiflow.lazy import KNNClassifier
from sklearn import svm

from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime

DATEPARSE = "%Y-%m-%d %H:%M:%S"

# Read data
### Add data into /taxi_data/
### Github stores a sample file of this. Download the rest and do the same.

In [15]:
def time_to_minutes(_today):
    return (_today.weekday()+1) * 24*60 + (_today.hour*60) + (_today.minute)
def add_zero_padding(number):
    if len(str(number)) == 1: return "0" + str(number)
    else: return number
def format_df(DF, pickup_date_column, dropof_date_column, month):
    for i, x in DF.iterrows():
        DF.at[i,"month"] = month
        start = datetime.strptime(x[pickup_date_column], DATEPARSE)
        stop = datetime.strptime(x[dropof_date_column], DATEPARSE)

        DF.at[i, pickup_date_column] = time_to_minutes(start)
        DF.at[i, dropof_date_column] = time_to_minutes(stop)
    return DF

In [16]:
#GREEN

"""
header = True
for x in range(1, 13): # this takes 10-15 min
    try:
        dfGreen = pd.read_csv("taxi_data\green_tripdata_2019-" + str(add_zero_padding(x)) + ".csv", usecols=["PULocationID", "DOLocationID", "lpep_pickup_datetime", "lpep_dropoff_datetime"])
        dfGreen = format_df(dfGreen, "lpep_pickup_datetime", "lpep_dropoff_datetime", x)
        dfGreen.to_csv("taxi_data\green_tripdata_2019_concat.csv", mode='a+', header=header, index=False)
        header=False
        print("GREEN: " + str(x))
    except Exception as f:
        print(f)
"""
# YELLOW
header = True
for x in range(1, 13): # 12 * 612 * 15
    try:
        dfYellow = pd.read_csv("taxi_data\yellow_tripdata_2019-" + str(add_zero_padding(x)) + ".csv", usecols=["PULocationID", "DOLocationID", "tpep_pickup_datetime", "tpep_dropoff_datetime"])
        dfYellow = format_df(dfYellow, "tpep_pickup_datetime", "tpep_dropoff_datetime", x)
        dfYellow.to_csv("taxi_data\yellow_tripdata_2019_concat.csv", mode='a+', header=header, index=False)
        header=False
        print("YELLOW: " + str(x))
    except Exception as f:
        print(f)

YELLOW: 1


KeyboardInterrupt: 

# Train

## Using chunksize (really slow)

In [2]:
dfGreen = pd.read_csv("taxi_data\green_tripdata_2019_concat.csv", chunksize=500000)

model = KNNClassifier(n_neighbors=50)

for chunk in dfGreen:
    X_green = chunk[["lpep_pickup_datetime", "PULocationID", "DOLocationID", "month"]] # input cols
    y_green = chunk[["lpep_dropoff_datetime"]] # output cols

    X_train, X_test, y_train, y_test = train_test_split(X_green.values, y_green.values, test_size = 0.25)

    model.partial_fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
    print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))


## Who needs memory lul

In [7]:
dfGreen = pd.read_csv("taxi_data\green_tripdata_2019_concat.csv")

model = KNeighborsClassifier(300)

X_green = dfGreen[["lpep_pickup_datetime", "PULocationID", "DOLocationID", "month"]] # input cols
y_green = dfGreen[["lpep_dropoff_datetime"]] # output cols

X_train, X_test, y_train, y_test = train_test_split(X_green.values, y_green.values, test_size = 0.25)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print("Mean squared error: %.2f" % mean_squared_error(y_test, y_pred))
print("Coefficient of determination: %.2f" % r2_score(y_test, y_pred))

C:\Python310\lib\site-packages\sklearn\neighbors\_classification.py:198: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


Mean squared error: 107251.78
Coefficient of determination: 0.99


In [11]:
"""
lpep_pickup_datetime,PULocationID,DOLocationID,month
"""
copied_from_green = [4284,4300,185,247,12.0]

inputdata = [copied_from_green[0], copied_from_green[2], copied_from_green[3], copied_from_green[4]]
outputdata = [copied_from_green[1]]

pred = model.predict([
    inputdata
])

print(pred)

[4286]
